In [15]:
%matplotlib widget
import os
import h5py
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import find_peaks
from scipy.optimize import curve_fit

from tqdm.notebook import tqdm

In [2]:
data_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_room2", "t2_v2")
json_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_room2", "ketek_room2_reprocs.json")
save_path = os.path.join("/Volumes", "TOSHIBA_EXT", "sipm_data_2", "ketek_box", "ketek_room2", "plots", "decay_constants")

In [3]:
file_list = glob.glob(f"{data_path}/*.h5")
file_list

['/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_25.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_26.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_27.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_28.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_29.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t2_v2/t2_ketek_room2_30.h5']

In [4]:
file_num = 3

In [32]:
def exp_charge(x, A, x0, tau):
    result = A*(1 - np.exp(-(x-x0)/tau))
    result[result < 0] = 0
    return result

def double_exp_release(x, x0, A1, A2, b1, b2):
    return A1*np.exp(-(x-x0)/b1) + A2*np.exp(-(x-x0)/b2)

def time_constants(times, one_waves, lback=10, lfor=500, height=500, distance=100):
    short_taus = []
    long_taus = []
    charge_taus = []
    for i, wave in tqdm(enumerate(one_waves), total=len(one_waves)):
        peak_info = find_peaks(wave, height=height, distance=distance, width=4)
        if (len(peak_info[0]) > 1) | (len(peak_info[0]) == 0): continue
        charge_time, release_time = times[peak_info[0][0]-lback:peak_info[0][0]], times[peak_info[0][0]:lfor]
        charge_form, release_form = wave[peak_info[0][0]-lback:peak_info[0][0]], wave[peak_info[0][0]:lfor]
        if (len(charge_form) == 0) | (len(release_form) == 0): continue
        try:
            release_coeffs, release_cov = curve_fit(double_exp_release, release_time, release_form, p0=[50, 200, 5e6, 500, 7])
            charge_coeffs, charge_cov = curve_fit(exp_charge, charge_time, charge_form, p0=[600, 92, 5])
            short_taus.append(min(release_coeffs[-1], release_coeffs[-2]))
            long_taus.append(max(release_coeffs[-1], release_coeffs[-2]))
            charge_taus.append(charge_coeffs[-1])
        except RuntimeError:
            continue
    return np.array(short_taus), np.array(long_taus), np.array(charge_taus)

In [6]:
h5_file = h5py.File(file_list[file_num], "r")
waveforms = h5_file["/processed/blr_wf"][:]
norm_charges = h5_file["/processed/norm_charge"][:]
one_waves = waveforms[(norm_charges > 0.90) & (norm_charges < 1.10)]

In [9]:
time = np.arange(0, h5_file["/processed/blr_wf"].shape[1])*h5_file["/raw/dt"][()]

In [12]:
plt.figure()
for i in range(0, 100):
    plt.plot(time[0:500], one_waves[i][0:500], alpha=0.15)
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (ADC)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Amplitude (ADC)')

In [33]:
shorts, longs, charge_ups = time_constants(time, one_waves)

<ipython-input-32-7a36c17fd79b>:7: RuntimeWarning: overflow encountered in exp
  return A1*np.exp(-(x-x0)/b1) + A2*np.exp(-(x-x0)/b2)
/usr/local/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
<ipython-input-32-7a36c17fd79b>:7: RuntimeWarning: overflow encountered in multiply
  return A1*np.exp(-(x-x0)/b1) + A2*np.exp(-(x-x0)/b2)
<ipython-input-32-7a36c17fd79b>:2: RuntimeWarning: overflow encountered in exp
  result = A*(1 - np.exp(-(x-x0)/tau))


In [39]:
fig, ax = plt.subplots(2, 1)
n, bins, patches = ax[0].hist(shorts[shorts < 1000], bins=1000, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[0])
n, bins, patches = ax[0].hist(shorts[shorts < 1000], bins=1000, histtype="step", color=sns.color_palette()[0])
ax[0].set_xlabel("Time Constant (ns)")
ax[0].set_ylabel("Counts")
ax[0].set_title("Short Decay")

n, bins, patches = ax[1].hist(longs[longs < 100], bins=1000, histtype="stepfilled", alpha=0.15, color=sns.color_palette()[1])
n, bins, patches = ax[1].hist(longs[longs < 100], bins=1000, histtype="step", color=sns.color_palette()[1])
ax[1].set_xlabel("Time Constant (ns)")
ax[1].set_ylabel("Counts")
ax[1].set_title("Long Decay")

plt.tight_layout()
plt.savefig(f"{save_path}/fit_constants.png", dpi=600)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
def write_to_json(json_name, h5_file_name, short_tau, long_tau):
    head, tail = os.path.split(h5_file_name)
    reproc_dict = None
    with open(json_name, "r") as json_file:
        reproc_dict = json.load(json_file)
    
    reproc_dict["processes"]["fit_waveforms"][tail]["wf_in"] = "/processed/blr_wf"
    reproc_dict["processes"]["fit_waveforms"][tail]["wf_out"] = "/processed/fit_wf"
    reproc_dict["processes"]["fit_waveforms"][tail]["short_tau"] = short_tau
    reproc_dict["processes"]["fit_waveforms"][tail]["long_tau"] = long_tau
    reproc_dict["processes"]["fit_waveforms"][tail]["charge_up"] = short_tau
    
    with open(json_name, "w") as json_file:
        json.dump(reproc_dict, json_file, indent=4)

In [60]:
for file in file_list:
    write_to_json(json_path, file, [5, 4.9, 5.1], [60, 59.9, 60.1])

In [45]:
h5_file.close()